In [1]:
%%bash
chia version
cdv --version
python --version

1.6.2.dev130
cdv, version 1.1.3
Python 3.10.8


In [2]:
# did is the encoded launcher_id
did = "did:chia:10hq8mukapz8lml97tgnnsfdv24t9u5mw6pnqfh64kwh6vljykyrsc4m38t"
launcher_id = decode_puzzle_hash(did)
print(launcher_id)

7dc07df2dd088ffdfcbe5a273825ac55565e536ed06604df55b3afa67e44b107


In [3]:
full_node_client = await get_full_node_client()

get_coin_records_by_parent_ids = full_node_client.get_coin_records_by_parent_ids

current_did_singleton_cr = await singleton_utils.get_last_singleton_coin_record(get_coin_records_by_parent_ids, launcher_id)
print("Current DID Coin Record:")
print(current_did_singleton_cr)

parent_coin_info = current_did_singleton_cr.coin.parent_coin_info
spent_height = current_did_singleton_cr.confirmed_block_index

puzzle_solution = await full_node_client.get_puzzle_and_solution(parent_coin_info, spent_height)
puzzle = puzzle_solution.puzzle_reveal.to_program()


await close_rpc_client(full_node_client)


Current DID Coin Record:
{'coin': {'amount': 1,
          'parent_coin_info': '0x8ef7c734bcc2789cc13d7766423733308a92f5a649476829694d646a34d13776',
          'puzzle_hash': '0x378a6be25d8afa17e119da04a7327c80112396999b7d545fd32cc0cc8e36f64f'},
 'coinbase': False,
 'confirmed_block_index': 7,
 'spent_block_index': 0,
 'timestamp': 1667881000}


In [4]:
curried_singleton_args = puzzle.uncurry()[1]

print("DID Inner Puzzle:")
did_inner_puzzle = curried_singleton_args.rest().first()
print_program(did_inner_puzzle)

print("\nSynthetic PK:")
curried_did_inner_args = did_inner_puzzle.uncurry()[1]
synthetic_pk = G1Element.from_bytes(hexstr_to_bytes(disassemble(curried_did_inner_args.at("frrfrfr"))))
print(synthetic_pk)

DID Inner Puzzle:
(a (q 2 (q 2 (i -65 (q 2 5 383) (q 2 (i (all (= (a 126 (c 2 (c 6143 ()))) 11) (> 23 ())) (q 4 (c 40 (c 383 ())) (c (c 52 (c 767 (c 383 (c (c 767 ()) ())))) (c (c 56 (c 12287 ())) (a 38 (c 2 (c 47 (c 23 (c 6143 (c 12287 (c 767 (c 1535 (c 3071 (q ()))))))))))))) (q 8)) 1)) 1) (c (q (((49 . 61) 73 . 70) (a . 51) 60 . 4) ((q . 1) 2 2 (i 5 (q 2 58 (c 2 (c 13 (c (sha256 42 (sha256 34 60) (sha256 42 (sha256 42 (sha256 34 50) 9) (sha256 42 11 (sha256 34 ())))) ())))) (q . 11)) 1) ((a (i 23 (q 2 (i 319 (q 4 (c 48 (c (sha256 (sha256 (a 54 (c 2 (c 5 (c 39 (c 575 (c 1343 (c 2879 ()))))))) (a 126 (c 2 (c (a 46 (c 2 (c 47 (c 95 (c 383 ()))))) ())))) 47) ())) (a 38 (c 2 (c 5 (c 11 (c 55 (c 47 (c 95 (c 447 (c 383 (c (+ 767 (q . 1)) ()))))))))))) (q 2 38 (c 2 (c 5 (c 55 (c 47 (c 95 (c 447 (c 383 (c 767 ())))))))))) 1) (q 2 (i (> 767 (- 11 (q . 1))) (q 4 (c 32 (c 383 (c 95 ()))) ()) (q 8)) 1)) 1) 11 23 (a 94 (c 2 (c 9 (c 47 (c (a 126 (c 2 (c (c 9 (c 11 29)) ()))) ()))))) 95) (c (q . 1)

In [ ]:
alice_fp = 3855208971
wallet_client = await get_synced_wallet_client(alice_fp)

message = "This is Alice's DID"
print(f"message:\n{message}")
message_bytes = bytes(message, "utf-8")

synthetic_sk = await wallet_utils.synthetic_pk_to_sk(wallet_client, alice_fp, synthetic_pk, 10)
await close_rpc_client(wallet_client)

print(f"\nsynthetic pk:\n{bytes(synthetic_pk).hex()}")
print(f"\nsynthetic sk:\n{bytes(synthetic_sk).hex()}")

signature = AugSchemeMPL.sign(synthetic_sk, message_bytes)
print(f"\nsignature:\n{signature}")
print(message_bytes.hex())

assert AugSchemeMPL.verify(
    synthetic_pk,
    message_bytes,
    signature
)
